In [ ]:
from dataloader.pair_dataset import PairDataset

from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import numpy as np
import os

DATASET_PATH = "dataloader/dataloader/data/full/SAMPLE_001"

os.path.exists(DATASET_PATH)

rtstruct_path1 = (
    DATASET_PATH + "/RS.1.2.246.352.221.53086809173815688567595866456863246500.dcm"
)
rtstruct_path2 = (
    DATASET_PATH + "/RS.1.2.246.352.221.46272062591570509005209218152822185346.dcm"
)


DATASET_PATH = "dataloader/dataloader/data/full/SAMPLE_002"

rtstruct_path1 = (
        DATASET_PATH +"/" + "/RS.1.2.246.352.221.4648407614349067913684963238765492101.dcm"
    )
rtstruct_path2 = (
        DATASET_PATH  +"/" +  "RS.1.2.246.352.221.5699426928750258714918816447807823764.dcm"
    )

dataset = PairDataset(rtstruct_path1, rtstruct_path2)

size = 128
size = 1024
dataset = PairDataset(rtstruct_path2, rtstruct_path1, (size, size))

In [ ]:
from tranformation_utils import *

In [ ]:
#  do pair dataset nahradit           from dataloader.rs_dataset import RSDataset
# y_cutoff = 90 * int(size / 128)

In [ ]:
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import SimpleITK as sitk

dataloader = DataLoader(dataset, batch_size=4, shuffle=False)

for batch in dataloader:
    print(f"1 Batch shape: {batch['item1']['ct'].shape}")
    display(batch["item2"]["ct"])
    print(f"2 Batch shape: {batch['item1']['ct'].shape}")

    ct_batch1 = batch["item1"]["ct"]
    ct_batch2 = batch["item2"]["ct"]
    z_positions = batch["item1"]["z_position"]

    y_cutoff = 500 * int(size / 128)

    print(f"CT batch shape: {ct_batch1.shape}")
    for i in range(ct_batch1.shape[0]):
        # Convert tensors to NumPy arrays
        ct_img1 = ct_batch1[i, 0].detach().cpu().numpy()  # [H, W]
        ct_img2 = ct_batch2[i, 0].detach().cpu().numpy()
        z_pos = z_positions[i].item() if hasattr(z_positions[i], "item") else z_positions[i]

        # Convert to SimpleITK images
        planning_ct_img = sitk.GetImageFromArray(ct_img1)
        cbct_img = sitk.GetImageFromArray(ct_img2)

        # Apply smoothing
        smoothed_planning = sitk.SmoothingRecursiveGaussian(planning_ct_img, sigma=2.0)
        smoothed_cbct = sitk.SmoothingRecursiveGaussian(cbct_img, sigma=2.0)
        
      #   smoothed_planning = planning_ct_img
      #    smoothed_cbct = cbct_img
        # Perform rigid registration
        global_transform = perform_rigid_registration_v2(smoothed_planning, smoothed_cbct)

        # Resample CBCT to align with Planning CT
        moving_image_global_aligned = sitk.Resample(
            smoothed_cbct,
            smoothed_planning,
            global_transform,
            sitk.sitkLinear,
            -1000,
            smoothed_cbct.GetPixelID()
        )

        # Convert registered images back to NumPy
        ct_img1_np = sitk.GetArrayFromImage(smoothed_planning)
        ct_img2_np = sitk.GetArrayFromImage(moving_image_global_aligned)

        # Apply cutoff
        ct_img1_np[y_cutoff:, :] = 0
        ct_img2_np[y_cutoff:, :] = 0

        # Thresholding
        ct_img1_bin = np.where(ct_img1_np > 0.2, 1, 0)
        ct_img2_bin = np.where(ct_img2_np > 0.2, 1, 0)

        # Difference mask
        diff_mask = (ct_img1_bin != ct_img2_bin).astype(np.uint8)

        # Red overlay for visualization
        overlay = np.stack(
            [diff_mask * 255, np.zeros_like(diff_mask), np.zeros_like(diff_mask)],
            axis=-1,
        )

        fig, axs = plt.subplots(1, 3, figsize=(15, 5))

        axs[0].imshow(ct_img1_bin, cmap="gray")
        axs[0].set_title(f"Item1 - Z: {z_pos} | Y Cutoff: {y_cutoff}")
        axs[0].axis("off")

        axs[1].imshow(ct_img2_bin, cmap="gray")
        axs[1].set_title(f"Item2 - Z: {z_pos} | Y Cutoff: {y_cutoff}")
        axs[1].axis("off")

        # Difference ratio
        num_diff_pixels = np.sum(diff_mask)
        num_white_pixels_img1 = np.sum(ct_img1_bin == 1)
        num_white_pixels_img2 = np.sum(ct_img2_bin == 1)

        if (num_white_pixels_img1 + num_white_pixels_img2) > 0:
            diff_ratio = num_diff_pixels / (
                (num_white_pixels_img1 + num_white_pixels_img2) / 2
            )
        else:
            diff_ratio = 0.0

        axs[2].imshow(ct_img1_bin, cmap="gray")
        axs[2].imshow(overlay, alpha=0.6)
        axs[2].set_title(f"Diffs (red) | Ratio: {diff_ratio:.4f}")
        axs[2].axis("off")

        print(
            f"Item {i} - Z: {z_pos} | Diff Pixels: {num_diff_pixels}, "
            f"White Pixels (img2): {num_white_pixels_img2}, Ratio: {diff_ratio:.4f}"
        )

        plt.tight_layout()
        plt.show()


NameError: name 'dataset' is not defined